In [ ]:
import re
import pandas as pd
from pykospacing import Spacing

import torch
from konlpy.tag import Kkma
from transformers import BertTokenizer, BertModel

In [ ]:
spacing = Spacing()

def preprocessing(text):
    text = text.lower()                                 
    text = re.sub(r'[^\w\s]', '', text)                
    text = spacing(text)                                
    return text

def get_stopwords():                                    
    stopwords = set()
    with open('../../../../../data/stopwords-ko.txt', 'r', encoding='utf-8') as file:
        for line in file:
            stopwords.add(line.strip())
    return stopwords

def morph_and_remove_stopwords(text, stopwords):        
    text = preprocessing(text)                          
    kkma = Kkma()                                       
    morphs = kkma.morphs(text)                          
    meaningful_words = [word for word in morphs if word not in stopwords]                   
    return ' '.join(meaningful_words)

In [ ]:
stopwords = get_stopwords() 
df = pd.read_excel('../../../../../data/filtered_news.xlsx')     

df['processed_feature'] = df['feature'].apply(lambda x: morph_and_remove_stopwords(x, stopwords))       # 'feature' 열에 대해 형태소 분석 및 불용어 제거
print("Processed features:\n", df['processed_feature'].head())

tokenizer = BertTokenizer.from_pretrained('monologg/kobert')                                            # 토크나이저 로드
model = BertModel.from_pretrained('monologg/kobert')                                                    # KoBERT 모델 로드


In [ ]:

# KoBERT 임베딩 추출 함수
def get_elctra_embedding(text, tokenizer, model):                                                      
    if not text.strip():                                                                               
        return [0] * model.config.hidden_size
    
    # 텍스트를 토큰화하고 텐서로 변환
    inputs = tokenizer(text, return_tensors='pt', padding=True, truncation=True, max_length=512)
    input_ids = inputs['input_ids']
    attention_mask = inputs['attention_mask']
    token_type_ids = inputs.get('token_type_ids')

    # 모델을 사용하여 입력 텐서를 처리
    with torch.no_grad():
        outputs = model(input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)

    # 첫 번째 토큰 [CLS]의 임베딩을 사용
    cls_embedding = outputs.last_hidden_state[:, 0, :].squeeze().numpy()                            
    return cls_embedding

df['feature_vector'] = df['processed_feature'].apply(lambda x: get_elctra_embedding(x, tokenizer, model))